In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

In [11]:
df_elec = pd.read_csv("D:\\Great Learning\\R6\\ratings_Electronics.csv",engine='python')

In [12]:
df_elec.shape

(7824481, 4)

In [13]:
df_elec.sample(5)

,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1025422,A29E3ENX3ALT1L,B000I2C80K,5.0,1291593600
1254295,A2IEEL19SN6CIJ,B000OKH75M,2.0,1378512000
2171260,A17ST4V54LHQL8,B001NJ0PBG,4.0,1354147200
4066913,AQB4DH57MSZSC,B004F9P2KU,5.0,1308700800
2325904,A2065HBMYDXJ1S,B001TOD7N8,5.0,1255564800


In [14]:
df_elec.columns = ['userid','productid','ratings','timestamp']

In [15]:
df_elec.head(5)

,userid,productid,ratings,timestamp
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600


In [16]:
df_elec_copy = df_rat.copy

In [17]:
df_elec = df_elec.drop('timestamp',axis=1)

In [18]:
df_elec.head(5)

,userid,productid,ratings
0,A2CX7LUOHB2NDG,0321732944,5.0
1,A2NWSAGRHCP8N5,0439886341,1.0
2,A2WNBOD3WNDNKT,0439886341,3.0
3,A1GI0U4ZRJA8WN,0439886341,1.0
4,A1QGNMC6O1VW39,0511189877,5.0


In [19]:
# selecting only those users who has given more than 50 number of ratings

In [20]:
counts = df_elec['userid'].value_counts()
df_elec_new = df_elec[df_rat['userid'].isin(counts[counts >= 50].index)]

In [22]:
df_elec_new.shape

(125871, 3)

In [23]:
df_elec_new['userid'].value_counts()

A5JLAU2ARJ0BO     520
ADLVFFE4VBT8      501
A3OXHLG6DIBRW8    498
A6FIAB28IS79      431
A680RUE1FDO8B     406
A1ODOGXEYECQQ8    380
A36K2N527TXXJN    314
A2AY4YUOX2N1BQ    311
AWPODHOB4GFWL     308
A25C2M3QF9G7OQ    296
ARBKYIVNYWK3C     296
A22CW0ZHY3NJH8    292
A38RMU1Y5TDP9     282
A3EXWV8FNSSFL6    282
A3LGT6UZL99IW1    279
A2NOW4U7W3F7RI    277
A23GFTVIETX7DS    270
A3PD8JD9L4WEII    266
A17BUUBOU0598B    261
A3AYSYSLHU26U9    257
A2XRMQA6PJ5ZJ8    253
A231WM2Z2JL0U3    252
A12DQZKRKTNF5E    252
A1UQBFCERIP7VJ    247
AGVWTYW0ULXHT     244
A203OCQQ12MAVT    240
AEJAGHLC675A7     239
A2NYK9KWFMJV4Y    238
A3A4ZAIBQWKOZS    236
A31N0XY2UTB25C    227
                 ... 
A1W4F91DH3XPB2     50
A274X54O9CN2SR     50
A2ZGDHLJ85T6AD     50
A1UNJ46NSB352E     50
AOTZOHB2LZGYR      50
ANEXFROORI6F6      50
A11R5VW7IM6ISQ     50
A37PV5GMP2ILJC     50
A3JAH5WW61N2EK     50
AXU8RH1DEV21H      50
A1IU4UAV9QIJAI     50
ACH055GTTIGC9      50
A20ZM879PCQQEZ     50
A1PS4OYWUB0VX      50
AMO1MLSIJS

Build a popularity recommendation model based on rank ordering. This will give recommendation based on popularity ranking only and will not be personalized

In [58]:
#Class for Popularity based Recommender System model
class popularity_recommender_py():    
    def __init__(self):
        self.train_data = None
        self.userid = None        
        self.productid = None        
        self.popularity_recommendations = None            
    #Create the popularity based recommender system model    
    def create(self, train_data, userid, productid):
        self.train_data = train_data
        self.userid = userid        
        self.productid = productid         
        
        #Get a count of user_ids for recommendation score
        train_data_grouped = train_data.groupby([self.productid]).agg({self.userid: 'count'}).reset_index()        
        train_data_grouped.rename(columns = {'userid': 'score'},inplace=True)            
        #Sort the products based upon recommendation score
        train_data_sort = train_data_grouped.sort_values(['score', self.productid], ascending = [0,1])            
        #Generate a recommendation rank based upon score
        train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first')
        #Get the top 10 recommendations
        self.popularity_recommendations = train_data_sort.head(10)     
        #Use the popularity based recommender system model to    
        #make recommendations    
    def recommend(self, userid):            
        user_recommendations = self.popularity_recommendations                 
        #Add user_id column for which the recommendations are being generated        
        user_recommendations['userid'] = userid            
        #Bring user_id column to the front        
        cols = user_recommendations.columns.tolist()        
        cols = cols[-1:] + cols[:-1]        
        user_recommendations = user_recommendations[cols]
        return user_recommendations

In [59]:
pm = popularity_recommender_py()

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
train_data, test_data = train_test_split(df_elec_new, test_size = 0.20, random_state=0)

In [62]:
train_data.shape

(100696, 3)

In [63]:
test_data.shape

(25175, 3)

In [64]:
users = df_elec_new['userid'].unique()
len(users) ## unique users

1540

In [65]:
products = df_elec_new['productid'].unique()
len(products) ## unique products

48190

In [66]:
pm.create(train_data, 'userid', 'productid')

In [ ]:
# user popularity model to make predictions --- top 10 recommendations

In [67]:
user_id = users[5]

In [68]:
pm.recommend(user_id)

,userid,productid,score,Rank
33726,A3TAS1AG6FMBQW,B0088CJT4U,160,1.0
21503,A3TAS1AG6FMBQW,B003ES5ZUU,146,2.0
33101,A3TAS1AG6FMBQW,B007WTAJTO,134,3.0
9568,A3TAS1AG6FMBQW,B000N99BBC,128,4.0
33402,A3TAS1AG6FMBQW,B00829TIEK,118,5.0
34011,A3TAS1AG6FMBQW,B008DWCRQW,113,6.0
33398,A3TAS1AG6FMBQW,B00829THK0,104,7.0
19040,A3TAS1AG6FMBQW,B002R5AM7C,103,8.0
19238,A3TAS1AG6FMBQW,B002SZEOLG,95,9.0
24884,A3TAS1AG6FMBQW,B004CLYEDC,95,10.0


In [70]:
train_data_grouped = train_data.groupby('productid').mean().reset_index()
train_data_sort = train_data_grouped.sort_values(['ratings', 'productid'], ascending = [0,1])

In [71]:
pred_df = test_data[['userid', 'productid', 'ratings']]
pred_df.rename(columns = {'ratings' : 'true_ratings'}, inplace=True)
pred_df = pred_df.merge(train_data_sort,left_on='productid', right_on = 'productid')
pred_df.rename(columns = {'ratings' : 'predicted_ratings'}, inplace = True)

In [72]:
import sklearn.metrics as metrics
from math import sqrt
MSE = metrics.mean_squared_error(pred_df['true_ratings'], pred_df['predicted_ratings'])

In [73]:
MSE

1.1800434652999245

In [ ]:
# RMSE for popularity model is as follows

In [74]:
sqrt(MSE)

1.0862980554617248

In [75]:
## collaborative filtering recommendetion system

In [76]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from surprise import Dataset,Reader

In [77]:
reader = Reader(rating_scale=(0.5, 5))

In [79]:
data = Dataset.load_from_df(df_rat_new[['userid', 'productid', 'ratings']], reader)d

In [80]:
data

In [81]:
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy

In [82]:
trainset, testset = train_test_split(data, test_size=.25,random_state=69)

In [102]:
svd_model = SVD(n_factors=150,reg_all = 0.01,lr_all=0.001)

In [103]:
svd_model.fit(trainset)

In [104]:
test_pred = svd_model.test(testset)

In [105]:
accuracy.rmse(test_pred)

RMSE: 1.0024


1.002440391137716

In [106]:
# use grid search for finding out the best hyerparameters

In [111]:
from surprise.model_selection import GridSearchCV

In [113]:
param_grid = {'lr_all': [0.001, 0.003, 0.005, 0.008],'reg_all': [0.08, 0.1, 0.15]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.9811834043524786
{'lr_all': 0.008, 'reg_all': 0.15}


In [114]:
svd_model_new = SVD(n_factors=150,reg_all = 0.15,lr_all=0.008)

In [185]:
svd_model_new.fit(trainset)

In [186]:
test_pred1 =  svd_model_new.test(testset)

In [187]:
# check RMSE after selecting the best hyper parameter on test set

In [188]:
accuracy.rmse(test_pred1)

RMSE: 0.9837


0.9836667482682849

In [189]:
user_factors = svd_model_new.pu
item_factors = svd_model_new.qi

In [190]:
pred_1 = np.dot(user_factors,np.transpose(item_factors))

In [163]:
df_pred = pd.DataFrame(test_pred1)

In [168]:
df_pred.shape

(31468, 5)

In [173]:
df_pred.columns

Index(['uid', 'iid', 'r_ui', 'est', 'details'], dtype='object')

In [181]:
df_pred_sorted = df_pred.sort_values(by=['uid','est'],ascending=[True,False]).groupby('uid').head(5)

In [182]:
df_pred_sorted

,uid,iid,r_ui,est,details
21637,A100UD67AHFODS,B00108UNEY,5.0,4.799345,{'was_impossible': False}
28160,A100UD67AHFODS,B009LL9VDG,5.0,4.741876,{'was_impossible': False}
23919,A100UD67AHFODS,B001W26TIW,5.0,4.703939,{'was_impossible': False}
23430,A100UD67AHFODS,B005PQTYHE,5.0,4.697746,{'was_impossible': False}
20485,A100UD67AHFODS,B00F0DD0I6,5.0,4.680470,{'was_impossible': False}
20813,A100WO06OQR8BQ,B001342KM8,5.0,4.313267,{'was_impossible': False}
1206,A100WO06OQR8BQ,B003ES5ZUU,5.0,4.278864,{'was_impossible': False}
14962,A100WO06OQR8BQ,B0096LZ5QI,5.0,4.100296,{'was_impossible': False}
29684,A100WO06OQR8BQ,B000HPV3RW,5.0,4.065618,{'was_impossible': False}
26999,A100WO06OQR8BQ,B00CGYCOR0,5.0,4.038950,{'was_impossible': False}


collaborative filtering gives personalized recommendation for products / items. This will have more chances of user buying the
product

Popularity model doesnt give personalized recommendation

In [147]:
from collections import defaultdict

In [148]:
# create a routine for getting top n (5) reco

In [151]:
def get_top_n(predictions, n):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
         top_n[uid].append((iid, est))
       

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [152]:
get_top_n(test_pred1,5)

defaultdict(list,
            {'A2KYRPDKVBI2R4': [('B000N99BBC', 5),
              ('B0013FRNKG', 5),
              ('B00APP6694', 4.98771658377799),
              ('B003X26PMO', 4.896130389660365),
              ('B00CXP5R6M', 4.862952372427997)],
             'A1GQRGB8FGSLIZ': [('B001OC6DCW', 4.466807944772594),
              ('B00EOE4G4E', 4.146807161811678),
              ('B00DEQLDDU', 4.104464126905625),
              ('B000M4XMGI', 4.100786893387462),
              ('B00008WFVL', 4.025748167209783)],
             'A6FIAB28IS79': [('B000F7JCRA', 4.660113727948097),
              ('B001FA1NZK', 4.62957173851362),
              ('B000FQ2JLW', 4.553627565975782),
              ('B0013FRNKG', 4.525260529244962),
              ('B000OFSN4G', 4.506696691937332)],
             'A1SX9PHJWTDMF9': [('B000O8I474', 4.913712480203411),
              ('B001SH2BDI', 4.827890765246331),
              ('B003LL51V8', 4.7120537186998375),
              ('B0007QCO6A', 4.706308314269422),
           